<a href="https://colab.research.google.com/github/pabrey/datasciencecoursera/blob/master/4b_all_routes_mean_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Previous

In [1]:
import pandas as pd
import io
import matplotlib.pyplot as plt
import numpy as np
import json
from google.colab import files
import time
from datetime import datetime
import math

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Create functions

## load data

In [0]:

def load_data(location):
  Separator = ';'

  df =  pd.read_csv(location, sep=Separator)

  df.columns = ["CASE_ID", "ACTIVITY",	"TIMESTAMP_IN",	"TIMESTAMP_OUT", "COLOR"] 
  df["DIFFERENCE"]= df["TIMESTAMP_OUT"] - df["TIMESTAMP_IN"]
  df_order = df.sort_values(['CASE_ID', 'TIMESTAMP_IN'])

  print(df_order.head())
  print(df_order.head(1))
  print(df_order.tail(1))
  print(df_order.tail(1))
  return df_order

## prefilter

thers elements with timestamp to 0, delete this elements

In [0]:
def filter_bad_data(df_input):
  #hers elements with timestamp to 0, delete this elements
  id_filter = df_input[df_input["TIMESTAMP_IN"] == 0]["CASE_ID"]
  df_filter_out = df_input[~df_input["CASE_ID"].isin(id_filter)]
  id_filter = df_input[df_input["TIMESTAMP_OUT"] == 0]["CASE_ID"]
  df_filter_out = df_filter_out[~df_filter_out["CASE_ID"].isin(id_filter)]
  df_filter_out = df_filter_out.sort_values(['CASE_ID', 'TIMESTAMP_IN'])
  
  #elementos no ordenados
  df_unorder = df_filter_out.sort_values("TIMESTAMP_IN").groupby("CASE_ID").tail(1)[["CASE_ID", "ACTIVITY"]] 
  id_unorder = df_unorder[df_unorder["ACTIVITY"] != "Aalta"]["CASE_ID"]
  df_filter_out = df_filter_out[~df_filter_out["CASE_ID"].isin(id_unorder)]
  df_filter_out = df_filter_out.sort_values(['CASE_ID', 'TIMESTAMP_IN'])
  df_filter_out = df_filter_out.sort_values(["CASE_ID", "TIMESTAMP_IN"])

  print("Len before prefilter: "+str(len(df_input["CASE_ID"].drop_duplicates())))
  print("Len after prefilter: "+str(len(df_filter_out["CASE_ID"].drop_duplicates())))
  
  return df_filter_out

## create minero

In [0]:

def create_minero_file(input_df, ouput_file):
  input_df = input_df[["CASE_ID", "ACTIVITY", "TIMESTAMP_IN"]]
  input_df = input_df.sort_values(["CASE_ID",  "TIMESTAMP_IN"])
  input_df["DATE_IN"] = (pd.to_datetime(input_df["TIMESTAMP_IN"]*1000,unit='ms'))
  input_df["TIMESTAMP"] = input_df["DATE_IN"].dt.strftime('%d/%m/%Y %H:%M:%S')+ " UTC"

  input_df[["CASE_ID","ACTIVITY","TIMESTAMP"]].sort_values([ "TIMESTAMP"]).to_csv(ouput_file, index=False, sep = ';')
  files.download(ouput_file)

# sinulation_00

## load and filter

In [5]:
location = 'drive/My Drive/data/prosae/testAllRoutesMean/res_simulacion_mean_00.csv'
df_load = load_data(location)


   CASE_ID        ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR    DIFFERENCE
4        1           Aalta  0.000000e+00   0.000000e+00    NaN  0.000000e+00
0        1        admision  1.614764e+08   1.614772e+08    NaN  8.395566e+02
1        1          triaje  1.614772e+08   1.614816e+08    NaN  4.409828e+03
2        1    rx solicitud  1.614816e+08   1.614831e+08    NaN  1.452231e+03
3        1  rx exploracion  1.614831e+08   0.000000e+00    NaN -1.614831e+08
   CASE_ID ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR  DIFFERENCE
4        1    Aalta           0.0            0.0    NaN         0.0
      CASE_ID  ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR    DIFFERENCE
1742      937  admision  1.620759e+08            0.0    NaN -1.620759e+08
      CASE_ID  ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR    DIFFERENCE
1742      937  admision  1.620759e+08            0.0    NaN -1.620759e+08


In [6]:
df_filter =  filter_bad_data(df_load)

Len before prefilter: 807
Len after prefilter: 801


## check elements whose last element is not alta and first is not admision

In [7]:
(df_filter.sort_values("TIMESTAMP_IN").groupby("CASE_ID")["ACTIVITY"].first() ).drop_duplicates()

CASE_ID
2    admision
Name: ACTIVITY, dtype: object

## show results

In [8]:
df_filter.groupby("ACTIVITY")["DIFFERENCE"].mean()

ACTIVITY
Aalta                 0.000000
admision            742.846072
interconsulta     20182.847068
respuesta         20819.107290
rx informe        12762.277320
rx solicitud      12611.087115
triaje           329974.480033
Name: DIFFERENCE, dtype: float64

Respuesta dada por java

System mean time: 380134.79391948384

admision : 743.8665863653418

alta : NaN

interconsulta : 19418.959930089968

respuesta : 22345.67648141086

rx exploracion : NaN

rx informe : 12483.702723689377

rx solicitud : 13521.27712738514

triaje : 328649.67725998274



In [9]:
len(df_filter)

1718

In [10]:
df_filter[df_filter["ACTIVITY"] == "admision"].head()

,CASE_ID,ACTIVITY,TIMESTAMP_IN,TIMESTAMP_OUT,COLOR,DIFFERENCE
5,2,admision,1.614772e+08,1.614780e+08,NaN,769.329164
9,4,admision,1.614780e+08,1.614786e+08,NaN,573.798124
12,5,admision,1.614786e+08,1.614804e+08,NaN,1863.717873
16,6,admision,1.614804e+08,1.614819e+08,NaN,1464.999379
20,9,admision,1.614819e+08,1.614821e+08,NaN,200.117493


## create file minero

In [0]:
ouput_file= 'input_data_miner_mean_00.csv'
create_minero_file(df_filter, ouput_file)

# sinulation_00_extende case

In [11]:
location = 'drive/My Drive/data/prosae/testAllRoutesMean/res_simulacion_mean_00_extended_time.csv'
df_load = load_data(location)

   CASE_ID        ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR    DIFFERENCE
0        1        admision  1.614764e+08   1.614772e+08    NaN  8.395566e+02
1        1          triaje  1.614772e+08   1.614816e+08    NaN  4.409828e+03
2        1    rx solicitud  1.614816e+08   1.614831e+08    NaN  1.452231e+03
3        1  rx exploracion  1.614831e+08   1.627340e+08    NaN  1.250893e+06
4        1      rx informe  1.627340e+08   1.627350e+08    NaN  1.034050e+03
   CASE_ID  ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR  DIFFERENCE
0        1  admision   161476390.0   1.614772e+08    NaN  839.556568
       CASE_ID  ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR    DIFFERENCE
17954     9931  admision  1.674754e+08            0.0    NaN -1.674754e+08
       CASE_ID  ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR    DIFFERENCE
17954     9931  admision  1.674754e+08            0.0    NaN -1.674754e+08


In [12]:
df_filter =  filter_bad_data(df_load)

Len before prefilter: 8025
Len after prefilter: 7994


## show results

In [14]:
df_filter.groupby("ACTIVITY")["DIFFERENCE"].mean()

ACTIVITY
Aalta             0.000000e+00
admision          7.479814e+02
interconsulta     1.289066e+05
respuesta         4.051206e+04
rx exploracion    3.103546e+06
rx informe        2.272065e+04
rx solicitud      3.303477e+04
triaje            2.805721e+06
Name: DIFFERENCE, dtype: float64

Teprico dado por java
admision : 747.6296508757738

alta : NaN

interconsulta : 127743.2180718071 -> 35 h

respuesta : 38876.30844513437 -> 10 h

rx exploracion : 3103545.5167867243 -> 862

rx informe : 23135.821598840495 -> 6.4 h

rx solicitud : 32551.43550627584 -> 9.1 h

triaje : 2808823.3745929995 -> 780.2 h



In [15]:
df_filter.head()

,CASE_ID,ACTIVITY,TIMESTAMP_IN,TIMESTAMP_OUT,COLOR,DIFFERENCE
0,1,admision,1.614764e+08,1.614772e+08,NaN,8.395566e+02
1,1,triaje,1.614772e+08,1.614816e+08,NaN,4.409828e+03
2,1,rx solicitud,1.614816e+08,1.614831e+08,NaN,1.452231e+03
3,1,rx exploracion,1.614831e+08,1.627340e+08,NaN,1.250893e+06
4,1,rx informe,1.627340e+08,1.627350e+08,NaN,1.034050e+03


In [19]:
df_filter[df_filter["TIMESTAMP_OUT"] == 0]

,CASE_ID,ACTIVITY,TIMESTAMP_IN,TIMESTAMP_OUT,COLOR,DIFFERENCE


## create minero

In [0]:
ouput_file= 'input_data_miner_mean_00_extended_time.csv'
create_minero_file(df_filter, ouput_file)